# A- Introduction and Business Problem

### A company who sells chocolate wants to enter a new market while segmenting coffee shops. This company wants to know the most recommended places in Canada to start a social media marketing campaign. The region has to be relatively small with a high number of coffee shops with good ratings.

### To provide the best recommendations for the chocolate company, we propose to choose a well-connected city with a relatively big population. Then, we will find the coffee shops with the highest frequencies and choose among them the best places depending on ratings.

### For the location, we proposed Toronto for the following reasons:
1-	Toronto is the capital of Ontario and is the most populated city in Canada (around 3 million in 2018). Toronto census metropolitan area (CMA) has a population of approximately 6 million. Therefore, it is Canada's most populous metropolis.

2-	Toronto is also an international center of “business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world” as per Wikipedia.

3-	Toronto area is interspersed with rivers, ravines and forests. Its current area is 630.2 km2.The city has a diverse population and is an important destination for immigrants to Canada.

4-	The city is a center for music, theatre, movie productionand television production. It contains cultural institutions like museums, galleries, festival and entertainment districts, national historic sites, and sports centers with over 43 million tourists each year.

5-	Toronto Stock Exchange (the headquarters of Canada's five largest banks)and multinational corporations are also located at Toronto. Also as per Wikipedia, “Its economy is highly diversified with strengths in technology, design, financial services, life sciences, education, arts, fashion, aerospace, environmental innovation, food services, and tourism.”

6-	Toronto is a great distribution point for the industrial sector. The city has a strategic position along the Quebec City–Windsor Corridor and has well connected infrastructure, roads and rails linking it to the surrounding cities.

### Therefore, as part of this project, we will list and visualize all coffee shops that would be our target in the social media marketing strategy for selling new chocolate bars in Toronto City. They will have the highest frequencies in their neighborhoods and good ratings.

---------

# B- Data:

### For this project we need to get the following data and start exploring it and we will download all the dependencies that we will need in the notebook:

1•	Toronto City data that contains lists of neighborhoods.We will rely on the postal codes to represent neighborhoods getting the table from Wikipedia:

    	To get their latitudes and longitudes, we will rely on the data source : https://cocl.us/Geospatial_data

    	Description: This data set contains the essential columns. 
         We will use it to explore various neighborhoods of Toronto city while focusing on coffee shops.



2•	Coffee shops in each neighborhood of Toronto city.

    	Data source : Foursquare API

    	Description: By using this API, we will get all the venues and coffee shops in each neighborhood. 
        We can filter these venues to get only the ones with the highest frequencies.



3•	Maps:

    	We will rely on Folium to get the maps and visualize the locations of the chosen shops.


In [5]:
#Getting the data
#Downloading all the dependencies

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


In [7]:
#Getting data and preparing it for analysis

In [8]:
# download data and parse it:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

#Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

# skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# make dataframe, expand rows and drop the old one:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

In [9]:
# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))


In [10]:
# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


In [11]:
#Getting more data: Geo-spatial data

dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)

toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [12]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geographical coordinates of Toronto, ON, Canada: {}, {}.'.format(latitude, longitude))

Geographical coordinates of Toronto, ON, Canada: 43.653963, -79.387207.


In [18]:
#We got the longitude and latitude
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Let's enter credentials for Foursquare to get the remaining data

In [21]:
CLIENT_ID = 'KLLVC0JYMWQ30TWJ3CAY0PZRZWLHMVDKOXWUNGAU4K4L1RWF' # my Foursquare ID
CLIENT_SECRET = 'CHSZ2PARKPP441LI333HATNCBUJ5DDFGEGLG3333S2HPZIQC' # your Foursquare Secret
VERSION = '20200301' # Foursquare API version

In [29]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name


## Here, we will make a query for coffee shops in a radius of 2000m

In [37]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=coffee'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KLLVC0JYMWQ30TWJ3CAY0PZRZWLHMVDKOXWUNGAU4K4L1RWF&client_secret=CHSZ2PARKPP441LI333HATNCBUJ5DDFGEGLG3333S2HPZIQC&v=20200301&ll=43.7532586,-79.3296565&radius=2000&limit=100&query=coffee'

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5d4cbc0de0d9001b8c8f3a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'coffee',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.77125861800002,
    'lng': -79.30478345939711},
   'sw': {'lat': 43.735258581999986, 'lng': -79.35452954060288}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57e286f2498e43d84d92d34a',
       'name': 'Tim Hortons',
       'location': {'address': '215 Brookbanks',
        'crossStreet': 'York Miils Rd',
        'lat': 43.76066827030228,
        'lng': -79.326367635

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tim Hortons,Café,43.760668,-79.326368
1,Baretto Caffé,Café,43.744456,-79.346460
2,Starbucks,Coffee Shop,43.754199,-79.351382
3,Aroma Espresso Bar,Coffee Shop,43.770880,-79.331775
4,Tim Hortons,Coffee Shop,43.741579,-79.318966


# As we can see in the table above, all categories are Café, Coffee shop, Coffee shops...

In [41]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


# Below, we will do the same to the others venues to get all coffee shops in the provided radius by Foursquare.

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=coffee'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park
Lawrence Manor, Lawrence Heights
Queen's Park 
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Ryerson, Garden District
Glencairn
West Deane Park, Cloverdale, Princess Gardens, Martin Grove, Islington
Rouge Hill, Highland Creek , Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Old Burnhamthorpe, Markland Wood
Morningside, Guildwood, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
King, Richmond, Adelaide
Dovercourt Village, Dufferin
Scarborough Village
Henry Farm, Oriole, Fairview
Northwood Park, York University
East Toronto
Union Station , Toronto Islands, Harbourfront East
Little Portugal, Trinity–Bellwoods
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
Riverd

In [49]:
print(toronto_venues.shape)
toronto_venues.head(2)

(944, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
1,Regent Park,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


## The size of the resulting dataframe is (944, 7)

In [50]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,2,2,2,2,2,2
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Downsview North, Wilson Heights",2,2,2,2,2,2
Bayview Village,1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",4,4,4,4,4,4
Berczy Park,23,23,23,23,23,23
"Bloordale Gardens, Eringate, Old Burnhamthorpe, Markland Wood",2,2,2,2,2,2
"Brockton, Parkdale Village, Exhibition Place",6,6,6,6,6,6
Business Reply Mail Processing Centre 969 Eastern,1,1,1,1,1,1


In [51]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 31 uniques categories.


# Now that we got our data, we will start the analysis in week 2 with one hot encoding, get dummies, frequencies and, finally, maps.